### Input Library

In [1]:
import tkinter as tk
import cv2
import numpy as np
import tensorflow as tf
from PIL import Image, ImageTk
from textblob import TextBlob
from spellchecker import SpellChecker
import threading
import pandas as pd

### Load Model 

In [2]:
model_path = "../models/final_model.h5"
loaded_model = tf.keras.models.load_model(model_path)

print("Model loaded successfully")

# List of letters according to model label
label_map = [chr(i) for i in range(65, 91)]  # A-Z

# Check the column names in your CSV file
df = pd.read_csv('../data/spelling/autocorrect_dataset.csv')

# Load the misspelled words
misspelled_words = df['misspelled'].tolist()

# Initialize SpellChecker without a specific language
spell = SpellChecker()

# Add custom misspelled words to the spell checker's word frequency list
spell.word_frequency.load_words(misspelled_words)



Model loaded successfully


In [3]:
# Function to preprocess the frame
def preprocess_frame(frame):
    img = cv2.resize(frame, (180, 180))
    img_array = np.array(img, dtype=np.float32) / 255.0
    img_array = np.reshape(img_array, (1, 180, 180, 3))
    return img_array

In [4]:
# Function to correct spelling and display suggestions
def correct_spelling(word):
    corrected_word = spell.correction(word)
    suggestions = spell.candidates(corrected_word)
    return corrected_word, suggestions


In [5]:
# Function to handle predictions
def predict(img):
    global current_symbol, word, blank_flag, str_text, ct, predicted_word, word_history

    try:
        result = loaded_model.predict(img)
        if result.shape[1] != 26:
            raise ValueError("Unexpected model output shape")

        prediction = {chr(65 + i): result[0][i] for i in range(26)}

        # Sort predictions in descending order
        prediction = sorted(prediction.items(), key=lambda x: x[1], reverse=True)
        current_symbol = prediction[0][0]
        confidence = prediction[0][1] * 100

        # Update letter count and word
        if current_symbol == " ":
            for i in ct:
                ct[i] = 0
        ct[current_symbol] += 1

        if ct[current_symbol] > 60:
            if current_symbol != " ":
                word += current_symbol
            ct = {chr(i): 0 for i in range(65, 91)}

        if current_symbol == " ":
            if blank_flag == 0:
                blank_flag = 1
                if len(str_text) > 0 and str_text[-1] != " ":
                    str_text += " "
                str_text += word
                word_history.append(word)
                word = ""
        else:
            blank_flag = 0

        # Correct the word and get suggestions in Indonesian
        corrected_word, suggestions = correct_spelling(word)

        # Update GUI with the predictions
        panel2.config(text=current_symbol, font=("Courier", 50))
        panel3.config(text=corrected_word, font=("Courier", 40))
        T.config(text=str_text, font=("Courier", 40))

        # Display history and suggestions
        print(f"Predicted Word: {corrected_word}")
        print(f"Word History: {word_history}")
        print(f"Suggestions: {', '.join(suggestions)}")

        return confidence

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return 0

In [6]:
# Declare img_tk as a global variable outside of the function
img_tk = None

def update_frame():
    global img_tk
    ret, frame = cap.read()
    if ret:
        frame = cv2.flip(frame, 1)

        # Define the central bounding box
        h, w, _ = frame.shape
        box_size = 300
        center_x, center_y = w // 2, h // 2
        x1, y1 = center_x - box_size // 2, center_y - box_size // 2
        x2, y2 = center_x + box_size // 2, center_y + box_size // 2

        # Draw the bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Extract the region of interest (ROI) for hand detection
        roi = frame[y1:y2, x1:x2]

        # Preprocess the ROI for prediction
        img_array = preprocess_frame(roi)
        confidence = predict(img_array)

        # Draw rectangle around detected face and put the confidence percentage
        cv2.putText(frame, f"{current_symbol}: {confidence:.2f}%", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img_tk = ImageTk.PhotoImage(image=img)

        # Update the Tkinter panel with the new image
        if panel.winfo_exists():
            panel.config(image=img_tk)
            panel.image = img_tk

    root.after(10, update_frame)


In [7]:
# Function to display history in a new window
def show_history():
    history_window = tk.Toplevel(root)
    history_window.title("Word History")
    history_text = tk.Text(history_window, font=("Courier", 20))
    history_text.pack()
    history_text.insert(tk.END, "\n".join(word_history))


In [8]:
# Function to display the final text in a new window
def show_final_text():
    final_window = tk.Toplevel(root)
    final_window.title("Final Text")
    final_text = tk.Label(final_window, text=str_text, font=("Courier", 40))
    final_text.pack()

In [9]:
# Initialize Tkinter
root = tk.Tk()
panel = tk.Label(root)
panel.pack()

panel2 = tk.Label(root, text="", font=("Courier", 50))
panel2.pack()
panel3 = tk.Label(root, text="", font=("Courier", 40))
panel3.pack()
T = tk.Label(root, text="", font=("Courier", 40))
T.pack()

history_button = tk.Button(root, text="Show History", command=show_history)
history_button.pack()

final_button = tk.Button(root, text="Show Final Text", command=show_final_text)
final_button.pack()

cap = cv2.VideoCapture(0)

# Initialize variables
current_symbol = " "
word = ""
blank_flag = 0
str_text = ""
ct = {chr(i): 0 for i in range(65, 91)}  # Dictionary to keep count of each letter
word_history = []  # History of words
predicted_word = ""  # The predicted word

update_frame()
root.mainloop()

# Release the video capture object and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
Predicted Word: i
Word History: []
Suggestions: i
1/1 ━━━━━━━━━━━━━━━━━━